In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

http://image-net.org/challenges/LSVRC/2014/browse-synsets

In [2]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions

In [3]:
winds = decode_predictions(np.expand_dims(np.zeros(1000), 0), top=1000)

In [4]:
import os
import requests
import shutil
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

In [5]:
def extension_mapper(ct):
    return ct.split('/')[-1]    

def download_image(ind, l, wind_path):
#     l = list_of_images[ind]
    try:
        r = requests.get(l, timeout=8)
        status_code = r.status_code
        if status_code == 200 and r.headers['Content-Type'] in image_formats:
            ext = extension_mapper(r.headers['Content-Type'])
            fname = '{}/{}.{}'.format(wind_path, ind, ext)

            response = {'ind':ind,
                        'url': l, 
                        'fname': fname,
                        'status_code': status_code}
            img_data = r.content
            with open(fname, 'wb') as f:
#                 r.raw.decode_content = True
#                 shutil.copyfileobj(r.raw, f)
                f.write(img_data)
        else:
            response = {'ind':ind,
            'url': l, 
            'fname': None,
            'status_code': status_code}
    except:
        response = {'ind':ind,
                    'url': l, 
                    'fname': None,
                    'status_code': 'err/to'}
    return response

In [6]:
data_path = '/data/datasets/imagenet2012/'
image_formats = ("image/png", "image/jpeg", "image/jpg")

In [ ]:
# wind = 'n02391049'
pool = Pool(23)
for wind in tqdm(winds[0]):
    wind = wind[0]
#     print(wind)
    wind_path = '{}/{}'.format(data_path, wind)
    os.makedirs(wind_path, exist_ok=True)
    resp = requests.get('http://www.image-net.org/api/text/imagenet.synset.geturls?wnid={wind}'.format(wind=wind))
    list_of_images = [x.replace('\n', '') for x in resp.text.split('\r')]
#     list_of_images = list_of_images[:20]
    inds = range(len(list_of_images))
    p_aux = [wind_path for x in inds]
    results = pool.starmap(download_image, zip(inds, list_of_images, p_aux))

    pd_results = pd.DataFrame(results)
    restuls_file = '{}/results.json'.format(wind_path)
    pd_results.to_json(restuls_file)

pool.close()

  8%|▊         | 81/1000 [2:33:27<38:44:08, 151.74s/it]

### brad images <3

PubFig: Public Figures Face Database

https://www.cs.columbia.edu/CAVE/databases/pubfig/download/

In [11]:
url = 'https://www.cs.columbia.edu/CAVE//databases/pubfig/download/eval_urls.txt'
r = requests.get(url)

In [23]:
list_of_c = [x.split('\t') for x in r.text.split('\n')]

In [27]:
list_of_c[1]

['#', 'person', 'imagenum', 'url', 'rect', 'md5sum']

In [28]:
list_of_c[2]

['Aaron Eckhart',
 '1',
 'http://farm1.static.flickr.com/119/288329997_19ebf1d7b3_o.jpg',
 '248,92,338,182',
 'a980a9e21c90ff62e57345fad53a56c8']

In [31]:
df = pd.DataFrame(list_of_c[2:-1], columns = list_of_c[1][1:] )
df_brad = df[df['person'] == 'Brad Pitt'] 

download images

In [37]:
wind = 'brad'

# wind = 'n02391049'
pool = Pool(23)

#     print(wind)
wind_path = '{}/{}'.format(data_path, wind)
os.makedirs(wind_path, exist_ok=True)

#     list_of_images = list_of_images[:20]
list_of_images = df_brad['url'].values
inds = range(len(list_of_images))
p_aux = [wind_path for x in inds]
results = pool.starmap(download_image, zip(inds, list_of_images, p_aux))

pd_results = pd.DataFrame(results)
restuls_file = '{}/results.json'.format(wind_path)
pd_results.to_json(restuls_file)

pool.close()